In [ ]:

# !pip install scikit-learn
# !pip install tensorflow
# !pip install seaborn

import tensorflow as tf
from tensorflow.keras import models, layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU, Dropout
import h5py
import numpy as np
from sklearn.metrics import confusion_matrix
import os
# DATASET_DIR = '../Dataset'
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 5

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/nutrient-deficiency-dataset/Final_dataset',
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    # class_names=['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']  # Replace with your own class names
)
print(len(dataset))
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    ds_size = len(ds)

    if(shuffle):
      ds = ds.shuffle(shuffle_size, seed=10)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)

    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
class_names = sorted(os.listdir('/kaggle/input/nutrient-deficiency-dataset/Final_dataset'))
print(len(test_ds))
# print(test_ds)
from keras.models import load_model
trained_model = load_model('/kaggle/input/nutrient-def-models/nutrient_def_model2.h5')
scores = trained_model.evaluate(test_ds)
print(scores)

predicted_labels = []
true_labels = []
for inputs, labels in test_ds:
#     print(inputs,labels,'\n')
    predicted = trained_model.predict(inputs)
    predicted_labels.extend(predicted.argmax(axis=-1))
    true_labels.extend(labels.numpy())

predicted_labels = [class_names[i] for i in predicted_labels]
true_labels = [class_names[i] for i in true_labels]
    
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import ConfusionMatrixDisplay

labels = np.array(class_names)
y_true = np.array(true_labels)
y_pred = np.array(predicted_labels)

cm = confusion_matrix(y_true, y_pred, labels=labels)

# Heat Map 
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
# plt.show()
plt.savefig('/kaggle/working/heat_map.png')

# Bar Chart
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred)
fig, ax = plt.subplots(figsize=(10, 6))  # set figure size for better visibility
x = np.arange(len(class_names))
width = 0.15  # reduce width of bars
ax.bar(x - width, precision, width, label='Precision')
ax.bar(x, recall, width, label='Recall')
ax.bar(x + width, f1, width, label='F1 score')
ax.set_xticks(x)
ax.set_xticklabels(class_names, rotation=45, ha='right')  # rotate and align xticklabels for better visibility
ax.set_xlabel('Class Names')
ax.set_ylabel('Scores')
ax.legend()
plt.tight_layout()  # adjust spacing between subplots
plt.savefig('/kaggle/working/bar_chart.png')

# Confusion Matrix with normalised values
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=class_names)
disp.plot(cmap="Blues")
# plt.show()
plt.tight_layout()
plt.savefig('/kaggle/working/conf_mat_normalised.png')


def precision_score(tp, fp):
    return tp / (tp + fp)
def recall_score(tp, fn):
    return tp / (tp + fn)
def f1_score(p, r):
    return 2 * p * r / (p + r)
def accuracy(tp, tn, fp, fn):
    return (tp + tn) / (tp + tn + fp + fn)

print('\n', *class_names)
print('\n', cm, '\n')
for i in range(len(class_names)):
    tp = cm[i, i]
    x = 0
    for j in range(len(class_names)):
        x += cm[j, i]
    fp = x - tp
    x = 0
    for j in range(len(class_names)):
        x += cm[i, j]
    fn = x - tp
    tn = len(true_labels) - tp - fp - fn

    print('TP = ',tp,' TN = ', tn,' FP = ', fp,' FN = ', fn)
    precision = precision_score(tp, fp)
    recall = recall_score(tp, fn)
    f1 = f1_score(precision, recall)
    _accuracy = accuracy(tp, tn, fp, fn)
    print(f'Class {class_names[i]}:\n'
          f'    Classification Accuracy  = {_accuracy:.4f}\n'
          f'    Precision = {precision:.4f}\n'
          f'    Recall    = {recall:.4f}\n'
          f'    F1-score  = {f1:.4f}\n')
